In [ ]:
# [cv] 얼굴인식하여 얼굴부분만 모자이크하기
# https://gwoolab.tistory.com/56

In [2]:
import cv2  # OpenCV 영상처리
import dlib   # https://updaun.tistory.com/entry/python-python-37-dlib-install-error
import numpy as np  # 배열 계산 용이
from imutils import face_utils, resize

In [5]:
# cap = cv2.VideoCapture('remove.avi')
cap = cv2.VideoCapture(0)  # 카메라 번호는 0부터 시작. 내장카메라는 0, 외장은 1부터. 컴퓨터에 내장 카메라가 아예 없는 경우, 연결된 외장 카메라는 0.
if not cap.isOpened():
    exit()

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
width = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 저장 파일명, 코덱, FPS, 크기 (width, height)
out = cv2.VideoWriter('mosaic_chim.avi', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()  # 카메라 상태 및 프레임

    if not ret:
        break

    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 흑백으로
    faces = detector(img_gray)
    for face in faces:
        landmarks = predictor(img_gray, face)
        points = face_utils.shape_to_np(landmarks)
        rect = cv2.boundingRect(points)
        x,y,w,h = rect

        # 얼굴 인식
        # cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 1)

        # 모자이크
        mosaic = frame[y:y+h, x:x+w].copy()
        mosaic = cv2.resize(mosaic, dsize=None, fx=0.1, fy=0.1, interpolation=cv2.INTER_NEAREST)
        mosaic = cv2.resize(mosaic, dsize=(w,h), interpolation=cv2.INTER_NEAREST)
        frame[y:y+h, x:x+w] = mosaic

    out.write(frame)
    cv2.imshow('camera', frame)
    if cv2.waitKey(1) == ord('q'):  # 사용자가 q를 입력하면
        break

out.release()
cap.release()
cv2.destroyAllWindows()